정확도 낮음  
제대로 학습이 되지 않음  
학습을 지속할 수록 감소 쪽으로 더 몰리거나 왔다가다 하는 경향
stride 커지면 관찰 기간 더 길어지면서 감소 쪽으로 더 몰리는 것도 같음
dilation 도 그럴 것 같지만 특별히 더 몰리지는 않음

## Load data

In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import matplotlib.pyplot as plt

In [3]:
import os, itertools
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import sys
#lib_s = '/home/adminuser/public/libs/pytorch_examples/basic_networks'
lib_s = '/home/bwlee/work/codes/examples/basic_networks'
sys.path.append(lib_s)
import generate_model as nets

## Make input data
interest.csv has only closed price.   
On the other hand, zipline needs OLHCV(?) format for each index  
make dummy columns and put 'close' value to them  
make csv file for each index  

In [4]:
f_s0 = 'interest.csv'
df0 = pd.read_csv(f_s0, index_col='date')
df0 = df0.iloc[::-1] # 과거에서 현재로 정렬순서 변경

## prepare data without time confusion
Seperate two actions

In [248]:
# normalized input, norm val, tgt_date, tgt_val, tgt_diff_ratio, 
class Environ():
    def __init__(self):
        self.indices = ['trea3', 'trea5', 'trea10']
        #self.tgt_index = 'trea10'
        self.tgt_index = 'trea3'
        # input window size
        self.input_size = 15
        # prediction is made after algo.pred_period from the present
        self.pred_period = 5

env = Environ() # setting values
mkt_data = {} # object to store data for learning
    
def push_data(env, data, mkt_data):
    """
    read past data and push them to mkt_data
    :param data: dataframe info from csv file
    :param mkt_data: dataframe to be used for learning
    """
    for ii_days in range(len(data)):
        if ii_days+1 < env.input_size:
            continue
        elif ii_days+1 == env.input_size:
            env.start_date = data.iloc[ii_days].name
        
        predicting_date = data.iloc[ii_days].name
        # historical data from today to env.input_size behind days
        history = data.iloc[ii_days+1-env.input_size:ii_days+1]
        mean_val = history.mean() 
        max_val = history.max()
        min_val = history.min()
        
        range_val = max_val - min_val
        norm_history = (history-mean_val)/range_val
        mkt_data[predicting_date] = [norm_history, mean_val, range_val, 
                                     None, None, None, None, None]
        
            
def push_tgt(env, data, mkt_data):
    """
    read future data and push them to mkt_data
    future_prediction_date is a future date when model want to predict
    past_predicting_date is a past date when model made a prediction with given information
    :param data: dataframe info from csv file
    :param mkt_data: dataframe to be used for learning
    """
    for ii_days in range(len(data)):
        ii_predicting = ii_days - env.pred_period
        predicting_date = data.iloc[ii_predicting].name
        if ii_predicting < 0 or predicting_date < env.start_date:
            continue
        
        past_history = mkt_data[predicting_date][0]
        mean_val = mkt_data[predicting_date][1]
        range_val = mkt_data[predicting_date][2]
        
        past_price = (past_history.iloc[-1]*range_val+mean_val)[env.tgt_index] # 49.6
        
        #past_price = (past_history.iloc[-5]*range_val+mean_val)[env.tgt_index] # 56.3
        
        #mkt_data[predicting_date][0] = past_history[:-4]
        
        #past_price = (past_history.iloc[-3]*range_val+mean_val)[env.tgt_index] # 53.0, # 53.0
        #past_price = (past_history.iloc[-13]*range_val+mean_val)[env.tgt_index] # 67.4
        #past_price = (past_history*range_val+mean_val).mean()[env.tgt_index] # 58.4
        #past_price = (past_history*range_val+mean_val)[:-3].mean()[env.tgt_index] # 63.2
        #past_price = (past_history*range_val+mean_val)[:-5].mean()[env.tgt_index] # 63.7
        
        #past_price = (past_history.iloc[-10]*range_val+mean_val)[env.tgt_index] # 66.4

        #past_price = (past_history.iloc[-5:].mean()*range_val+mean_val)[env.tgt_index] # 48.1
        #past_price = (past_history.iloc[-3:].mean()*range_val+mean_val)[env.tgt_index] # 48.1
        
        #past_price = (past_history.iloc[-10:].mean()*range_val+mean_val)[env.tgt_index] # 55.9
        #past_price = (past_history.iloc[-10:-5].mean()*range_val+mean_val)[env.tgt_index] # 60.4
        
        
        prediction_date = data.iloc[ii_days].name
        tgt_price = data.iloc[ii_days][env.tgt_index]
        #tgt_price = (data.iloc[ii_days-1:ii_days+2].mean())[env.tgt_index]
        
        tgt_ratio = (tgt_price - past_price)/past_price*10
        
        mkt_data[predicting_date][3] = prediction_date
        mkt_data[predicting_date][4] = np.single(tgt_price)
        mkt_data[predicting_date][5] = np.single(tgt_ratio)
        
        mkt_data[predicting_date][6] = np.single(past_price)
        mkt_data[predicting_date][7] = predicting_date

### Push past data

In [249]:
push_data(env, df0, mkt_data)

### Check data are pushed correctly
* no dates should be ahead of key date
* prediction info are not pushed yet

In [30]:
print(list(mkt_data.keys())[:3])

['2000-11-11', '2000-11-13', '2000-11-14']


In [523]:
mkt_data['2000-11-11']

[tensor([[ 0.3000,  0.2721,  0.2940],
         [ 0.1060,  0.1314,  0.1522],
         [ 0.1060,  0.1314,  0.1522],
         [ 0.0910,  0.1240,  0.1522],
         [ 0.1507,  0.1758,  0.1837],
         [ 0.1060,  0.1240,  0.1365],
         [ 0.1284,  0.1240,  0.1050],
         [ 0.1582,  0.1610,  0.1365],
         [ 0.1582,  0.1610,  0.1365],
         [ 0.1881,  0.1980,  0.1759],
         [ 0.1881,  0.2054,  0.1759],
         [ 0.0761,  0.0721,  0.0499],
         [-0.3567, -0.4242, -0.4383],
         [-0.7000, -0.7279, -0.7060],
         [-0.7000, -0.7279, -0.7060]]),
 tensor([7.6640, 7.8663, 8.1433]),
 tensor([0.6700, 0.6750, 0.6350]),
 '2000-11-17',
 7.275,
 0.004584002,
 7.2716665,
 '2000-11-11',
 tensor([-0.8881, -0.9333, -0.8819, -0.8582, -0.8889, -0.8425, -0.8060, -0.8519,
         -0.8425, -0.3433, -0.3037, -0.2677, -0.5149, -0.5852, -0.5748, -0.4627,
         -0.5481, -0.5748,  0.0299,  0.0444,  0.0394,  0.0821,  0.0815,  0.0394])]

In [209]:
env.start_date

'2000-11-06'

### Push future data

In [250]:
push_tgt(env, df0, mkt_data)
mkt_data = {key: val for key, val in mkt_data.items() if (val[-1] is not None)}

### CHECK tgt push is correctly made
end value of '2000-11-23' * norm == prediction value 

In [226]:
print(mkt_data['2000-11-11'])
print(mkt_data['2000-11-23'])

[               trea3     trea5    trea10
date                                    
2000-10-26  0.300000  0.272099  0.293963
2000-10-27  0.105970  0.131358  0.152231
2000-10-28  0.105970  0.131358  0.152231
2000-10-30  0.091045  0.123951  0.152231
2000-10-31  0.150746  0.175802  0.183727
2000-11-01  0.105970  0.123951  0.136483
2000-11-02  0.128358  0.123951  0.104987
2000-11-03  0.158209  0.160988  0.136483
2000-11-04  0.158209  0.160988  0.136483
2000-11-06  0.188060  0.198025  0.175853
2000-11-07  0.188060  0.205432  0.175853
2000-11-08  0.076119  0.072099  0.049869
2000-11-09 -0.356716 -0.424198 -0.438320
2000-11-10 -0.700000 -0.727901 -0.706037
2000-11-11 -0.700000 -0.727901 -0.706037, trea3     7.664000
trea5     7.866333
trea10    8.143333
dtype: float64, trea3     0.670
trea5     0.675
trea10    0.635
dtype: float64, '2000-12-05', 7.07, -1.0108074]
[               trea3     trea5    trea10
date                                    
2000-11-07  0.681934  0.687143  0.711795
2000-11-

In [1]:
## Training
### Data preparation

## Data preparation for Conv1d
In RNN, inputs are given as (time, indices)  
In Conv1d, indices are given as channel, which need to (channel: indices, features: time)  
(batch, channel=indices, data_size=seq_size)

In [254]:
class Index_data(Dataset):
    def __init__(self, in_data, dates):
        """
        :param in_data: dict of (df_past_normed, norm_val, prediction_date, tgt_price, tgt_ratio)
        """
        # change dataframe to float numpy array
        for key, val in in_data.items():
            if not 'Tensor' in str(type(in_data[key][0])) :
                in_data_temp = torch.tensor(in_data[key][0].to_numpy(np.single))
                print('in_temp', in_data_temp.size())
                in_data[key][0] = torch.tensor(np.transpose(in_data[key][0].to_numpy(np.single)))
                in_data[key][1] = torch.tensor(np.transpose(in_data[key][1].to_numpy(np.single)))
                in_data[key][2] = torch.tensor(np.transpose(in_data[key][2].to_numpy(np.single)))
                ## add difference
                diff = in_data_temp[-1] - in_data_temp[-5]
                diff2 = in_data_temp[-1] - in_data_temp[-7]
                diff3 = in_data_temp[-1] - in_data_temp[-10]
                diff4 = in_data_temp[-1] - in_data_temp[-3]
                diff5 = in_data_temp[-3] - in_data_temp[-7]
                diff6 = in_data_temp[-3] - in_data_temp[-10]
                diff7 = in_data_temp[-5] - in_data_temp[-7]
                diff8 = in_data_temp[-5] - in_data_temp[-10]
                #diff = torch.cat((diff, diff2, diff3), dim=-1)
                diff = torch.cat((diff, diff2, diff3, diff4, diff5, diff6, diff7, diff8), dim=-1)
                #print('in_temp2', in_data_temp.size(), 'diff', diff.size(), diff[:3])
                in_data[key].append(diff)
        self.in_data = in_data
        self.i2dates = {}
        for ii, date1 in enumerate(dates):
             self.i2dates[ii] = date1
        
    def __getitem__(self, ii_date):
        """
        :return: (df_past_normed, norm_val, prediction_date, tgt_price, tgt_ratio)
        """
        predicting_date = self.i2dates[ii_date]
        return self.in_data[predicting_date]
    
    def __len__(self):
        return len(self.in_data)

In [255]:
#erase data that do not have prediction target
traintest_dates = sorted(list(mkt_data.keys()))
n_train = len(traintest_dates)*8//10
n_test = len(traintest_dates) - n_train
train_dates = traintest_dates[:n_train]
test_dates = traintest_dates[n_train:]

train_data = {}
test_data = {}
for key, val in mkt_data.items():
    if key in train_dates:
        train_data[key] = val
    elif key in test_dates:
        test_data[key] = val

In [252]:
test_data['2021-01-15']

[            trea3  trea5    trea10
 date                              
 2020-12-24 -0.296  0.180 -0.375758
 2020-12-28 -0.196 -0.145 -0.648485
 2020-12-29  0.264  0.230 -0.284848
 2020-12-30  0.064  0.430  0.169697
 2020-12-31  0.064  0.430  0.169697
 2021-01-04 -0.236  0.180  0.351515
 2021-01-05 -0.596 -0.570 -0.321212
 2021-01-06 -0.136  0.105  0.351515
 2021-01-07  0.064  0.105  0.133333
 2021-01-08  0.204  0.080  0.133333
 2021-01-11 -0.036 -0.570 -0.503030
 2021-01-12  0.204 -0.170  0.078788
 2021-01-13  0.164 -0.270  0.078788
 2021-01-14  0.404  0.180  0.351515
 2021-01-15  0.064 -0.195  0.315152,
 trea3     0.966800
 trea5     1.317800
 trea10    1.712667
 dtype: float64,
 trea3     0.050
 trea5     0.040
 trea10    0.055
 dtype: float64,
 '2021-01-22',
 0.99,
 0.20618556,
 0.97,
 '2021-01-15']

In [256]:
batch_size = 64
train_dset = Index_data(train_data, train_dates)
test_dset = Index_data(test_data, test_dates)
train_loader = DataLoader(train_dset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dset, batch_size=batch_size, shuffle=False)

In [112]:
type(test_dset[0][-1])

torch.Tensor

In [14]:
print(len(test_dset))

1070


In [116]:
for ii in range(len(train_dset)):
    if type(train_dset[ii][-1]) is not torch.Tensor:
        print(train_dset[ii])

In [658]:
class Predictor(nets.Net):
    def __init__(self, net=None, downnet=None, drop=0.35,
                 loss=None, optimizer=None, device='cuda:2',
                 dim_hiddens=[100,200,300,50], seq_size = 15, n_kernels=[3,4,5], 
                 hidden_out=None, down_hiddens=[1, 1], add_down=-1,
                 dilations=None, pads=None, strides=None):
        """
        net is consist of [embed, net, downnet]
        dilation will be treated in another class
        initial seq_width, which corresponds to kernel_size, is 15(time window size)
        :param downnet: define downstream job
        :down_hiddens: multiply out_dimension of convolution
        """
        assert len(dim_hiddens)-1 == len(n_kernels)
        if dilations is None:
            dilations = [1 for _ in range(len(dim_hiddens))]
        if pads is None:
            pads = [0 for _ in range(len(dim_hiddens))]
        if strides is None:
            strides = [1 for _ in range(len(dim_hiddens))]
        
        super(Predictor, self).__init__(loss=loss, device=device)
        
        if net is None:
            self.convs = []
            relu = nn.ReLU()
            dropout = nn.Dropout(drop)
            # we do not care padding, stride, dilation
            out_seq_size = seq_size
            for ii, dim_hidden in enumerate(dim_hiddens):
                if ii==0:
                    continue
                
                n_kernel = n_kernels[ii-1]
                in_channel = dim_hiddens[ii-1]
                out_channel = dim_hiddens[ii]
                stride = strides[ii-1]
                pad = pads[ii-1]
                dilation = dilations[ii-1]
                
                conv1 = nn.Conv1d(in_channels=in_channel, 
                                  out_channels=out_channel,
                                  kernel_size=n_kernel,
                                  stride=stride,
                                  padding=pad,
                                  dilation=dilation
                                 )
                       
                out_seq_size0 = out_seq_size
                out_seq_size = (out_seq_size + 2*pad - dilation*(n_kernel-1)-1)//stride + 1
                #print(n_kernel, 'in', out_seq_size0, 'out', out_seq_size)
                self.convs.append(conv1)
            
            last_n_channel = dim_hiddens[-1]
            self.hidden_out =  last_n_channel * out_seq_size
            conv_nets = []
            for conv1 in self.convs:
                conv_nets.append(dropout)
                conv_nets.append(conv1)
                conv_nets.append(relu)
            self.net = nn.Sequential(*conv_nets)
        else:
            assert hidden_out is not None
            self.hidden_out = dim_out
            self.net = net
            
        if downnet is None:
            down_dims = [self.hidden_out*x for x in down_hiddens] + [1]
            if add_down>0:
                down_dims[0] += add_down
            self.downnet = nets.get_MLP(down_dims, dropout=0.35, end=True)
        else:
            self.downnet = downnet
        
        self.init_weights()
        
        self.net = self.net.to(device)
        self.downnet = self.downnet.to(device)
        
        parms = list(self.net.parameters())
        parms += list(self.downnet.parameters())
        self.optimizer = optimizer(parms)
        
    def set_train(self):
        self.net.train()
        self.downnet.train()
        
    def set_eval(self):
        self.net.eval()
        self.downnet.eval()
    
    def init_weights(self):
        for ii, layer in enumerate(self.net):
            if 'Conv' in str(layer):
                torch.nn.init.xavier_uniform_(self.net[ii].weight)
        for ii, layer in enumerate(self.downnet):
            if 'Linear' in str(layer):
                torch.nn.init.xavier_uniform_(self.downnet[ii].weight)
                #torch.nn.init.xavier_normal_(self.downnet[ii].weight)

    def forward(self, x, diff=None):
        out = self.net(x)
        dim_batch = out.shape[0]
        out = out.reshape(dim_batch, -1) # concatenate all data into 1D data
        #print('out size', out.size(), 'diff size', diff.size())
        if diff is not None:
            out = torch.cat((out, diff),dim=1)
        out = self.downnet(out) 
        return out.view(-1)

    def run_eval(self, data):
        self.set_eval()
        loss = 0
        outs = None
        tgts = None
        with torch.no_grad():
            for i_batch, data_batch in enumerate(data):
                data_in = data_batch[0]
                mean_val = data_batch[1]
                range_val = data_batch[2]
                pred_date = data_batch[3]
                tgt = data_batch[5]
                diff = data_batch[-1]
                
                data_in = data_in.to(self.device)
                tgt = tgt.to(self.device)
                diff = diff.to(self.device)
                out = self.forward(data_in, diff)
                #print('loss', loss, self.loss)
                loss += self.loss(out, tgt).cpu().numpy()
                out = out.cpu().numpy()
                #print('loss===', loss)
                tgt = tgt.cpu().numpy()
                if outs is None:
                    outs = out
                    tgts = tgt
                else:
                    outs = np.concatenate((outs, out), axis=0)
                    tgts = np.concatenate((tgts, tgt), axis=0)
        loss /= 1.0*(i_batch+1)
        print('evaluate', 'loss', loss, 'accuracy : define function')
        return outs, tgts, loss
    
    def run_batch(self, i_batch, data_batch):
        self.optimizer.zero_grad()
        data_in = data_batch[0]
        mean_val = data_batch[1]
        range_val = data_batch[2]
        pred_date = data_batch[3]
        tgt = data_batch[5]
        diff = data_batch[-1]
        #for val in diff:
        #    if type(val) is not torch.Tensor:
        #        print(diff)
        data_in = data_in.to(self.device)
        tgt = tgt.to(self.device)
        #print('diff-----------', diff)
        diff = diff.to(self.device)
        out = self.forward(data_in, diff)
        
        loss = self.loss(out, tgt)
        loss.backward()
        self.optimizer.step()
        return loss.detach().cpu().item()

In [727]:
seq_size = train_dset[0][0].shape[-1]
# Add difference variables
n_features = train_dset[0][0].shape[0]
down_hiddens = [1,2,2]
n_kernels=[3,3,3]
#dilations=[1,2,2]
#strides=[1,2,3]
strides=[1,2,2]
dim_hiddens = [n_features] + [50, 50, 50]
loss = nn.MSELoss() # which combines logsoftmax and nll loss
optimizer = optim.Adam
device = torch.device("cuda:2")

In [728]:
predictor = Predictor(loss=loss, optimizer=optimizer, 
                      device=device, seq_size= seq_size,
                      dim_hiddens=dim_hiddens, down_hiddens=down_hiddens, 
                      n_kernels=n_kernels, add_down=3*8,
                      strides=strides,
                      dilations=dilations,
                     )
                      #dilations=dilations)

In [757]:
predictor.run_train(n_epoch=10, data=train_loader)

epoch 0 loss 0.09203722945122576
eval_train evaluate loss 0.09066397496568623 accuracy : define function
epoch 1 loss 0.09190750016427751
eval_train evaluate loss 0.09029556427206566 accuracy : define function
epoch 2 loss 0.09171775250292535
eval_train evaluate loss 0.08994335914725687 accuracy : define function
epoch 3 loss 0.09239515145100764
eval_train evaluate loss 0.09006188353940622 accuracy : define function
epoch 4 loss 0.09180346031242342
eval_train evaluate loss 0.09027221443048164 accuracy : define function
epoch 5 loss 0.09100315041506468
eval_train evaluate loss 0.0896626860459349 accuracy : define function
epoch 6 loss 0.09199910744357465
eval_train evaluate loss 0.09026742220592143 accuracy : define function
epoch 7 loss 0.09119554981589317
eval_train evaluate loss 0.08967764119603741 accuracy : define function
epoch 8 loss 0.09149711412280354
eval_train evaluate loss 0.08981080553424892 accuracy : define function
epoch 9 loss 0.09211227186580202
eval_train evaluate los

In [758]:
rets = {}
tgts = None
preds = None
with torch.no_grad():
    for data_batch in test_loader:
        data_in = data_batch[0]
        tgt = data_batch[5]
        diff = data_batch[-1]
        
        data_in = data_in.to(device)
        diff = diff.to(device)
        tgt = tgt.numpy()
        #out = predictor.forward(data_in).cpu().numpy()
        out = predictor.forward(data_in, diff).cpu().numpy()
        if preds is None:
            preds = out
            tgts = tgt
        else:
            preds = np.concatenate((preds, out), axis=0)
            tgts = np.concatenate((tgts, tgt), axis=0)
            
        prediction_dates = data_batch[3]
        past_prices = data_batch[6]
        predicting_dates = data_batch[7]
        for ii, date1 in enumerate(prediction_dates):
            out1 = out[ii]
            out_orig = out1/10*past_prices[ii] + past_prices[ii]
            rets[date1] = {'predicting_date': predicting_dates[ii],
                          'pred': out_orig,
                           'past': past_prices[ii],
                           'tgt': data_batch[4][ii]}
            
preds = preds.reshape(-1)
tgts = tgts.reshape(-1)

In [759]:
sum1 = np.sum((preds > 0) & (tgts > 0))
sum2 = np.sum((preds < 0) & (tgts < 0))
print(sum1, np.sum(tgts>0))
print(sum2, np.sum(tgts<0))
print(len(preds), sum1+sum2, 1.0*(sum1+sum2)/len(preds))

78 482
495 570
1070 573 0.5355140186915888


In [760]:
limit = list(rets.keys()).index('2021-01-04')
sum1 = np.sum((preds[limit:] > 0) & (tgts[limit:] > 0))
sum2 = np.sum((preds[limit:] < 0) & (tgts[limit:] < 0))
print(sum1, np.sum(tgts[limit:]>0))
print(sum2, np.sum(tgts[limit:]<0))
print(len(preds[limit:]), sum1+sum2, 1.0*(sum1+sum2)/len(preds[limit:]))

11 44
19 24
71 30 0.4225352112676056


In [ ]:
list(preds)

## all includes 3*3=9 diff, predict 4 days avg, prediction_period=10

lstm_layer=2  
epoch 30, dropout=0, dim_hidden=100, mlp (dim_hidden, dim_hidden, 1) --> 55.5%  

epoch 30, dropout=0, dim_hidden=200, mlp (dim_hidden, dim_hidden, 1) --> 55.3%  

epoch 30, dropout=0, dim_hidden=200, mlp (dim_hidden, dim_hidden*2, 1) --> 53.4%  


epoch 30, dropout=0, dim_hidden=200, mlp (dim_hidden, dim_hidden//2, 1) --> 53.4%  
epoch 20, dropout=0, dim_hidden=200, mlp (dim_hidden, dim_hidden//2, 1) --> 53.9%  

epoch 20, dropout=0, dim_hidden=200, mlp (dim_hidden, dim_hidden, dim_hidden, 1) --> 48.8%  
epoch 30, dropout=0, dim_hidden=200, mlp (dim_hidden, dim_hidden, dim_hidden, 1) --> 51.4%  
epoch 40, dropout=0, dim_hidden=200, mlp (dim_hidden, dim_hidden, dim_hidden, 1) --> 53.7%  
epoch 50, dropout=0, dim_hidden=200, mlp (dim_hidden, dim_hidden, dim_hidden, 1) --> 49.1%  
epoch 60, dropout=0, dim_hidden=200, mlp (dim_hidden, dim_hidden, dim_hidden, 1) --> 48.7%  
epoch 70, dropout=0, dim_hidden=200, mlp (dim_hidden, dim_hidden, dim_hidden, 1) --> 50.3%  


epoch 10, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 52.2%  
epoch 20, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 52.0%  
epoch 30, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 56.9%, 54.3%  
epoch 40, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 54.9%  
epoch 50, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 53.3%  
epoch 60, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 48.6%, 49.6  

epoch 30, dropout=0, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 55.2  
epoch 40, dropout=0, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 55.8  
epoch 50, dropout=0, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 49.8  

epoch 30, dropout=0, dim_hidden=500, mlp (dim_hidden, dim_hidden, 1) --> 55.8  
epoch 40, dropout=0, dim_hidden=500, mlp (dim_hidden, dim_hidden, 1) --> 53.8  

lstm_layer=1
epoch 20, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 52.3%  
epoch 30, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 54.7%  

lstm_layer=3
epoch 30, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 55.7%  
epoch 40, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 55.4%  
epoch 50, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 53.8%  

## includes 3*8=24 diff
layer=2
epoch 20, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 52.5%  
epoch 30, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 54.3%  
epoch 40, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 53.9%  

## includes 3*8=24 diff, 1day prediction
layer=2
epoch 30, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 53.8%  

Below prediction days are (-2,-1,0)
## includes 3*8=24 diff, 3day prediction
layer=2
epoch 30, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 53.7%  

## includes 3*8=24 diff, 3day predicting, 3day prediction
layer=2
epoch 30, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 58.5%  

## includes 3*8=24 diff, 3day predicting, 3day prediction, prediction_period=5
layer=2
epoch 10, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 58.8%  
epoch 20, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 62.6%  
epoch 30, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 58.5%  
epoch 30, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 58.3%  

## includes 3*8=24 diff, 3day predicting, 3day prediction, prediction_period=15
layer=2
epoch 30, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 54.3%  

## includes 3*8=24 diff, 3day predicting, 3day prediction, prediction_period=20
layer=2
epoch 30, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 54.7%  
epoch 40, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 55.1%  
epoch 50, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 55.1%  
epoch 60, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 51.4%  

##  includes 3*8=24 diff, last price, 3day prediction, layer=2
epoch 5, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 51.7%  
epoch 10, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 56.4%, 54.8, 54.6  
epoch 10, dropout=0.35, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 52.7  
epoch 20, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 55.5%, 53.9 
epoch 20, dropout=0.35, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 54.3
epoch 30, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 49.3% 
epoch 30, dropout=0.35, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 55.2% 
epoch 40, dropout=0.35, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 54.5% 

epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 54.3  
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 54.5  
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 54.3  
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 57.3  
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 53.9  

## Bidirectional
epoch 10, dropout=0., dim_hidden=400, mlp (dim_hidden*2, dim_hidden*2, 1) --> 55.9 
epoch 20, dropout=0., dim_hidden=400, mlp (dim_hidden*2, dim_hidden*2, 1) --> 55.2 
epoch 30, dropout=0., dim_hidden=400, mlp (dim_hidden*2, dim_hidden*2, 1) --> 52.5 
epoch 40, dropout=0., dim_hidden=400, mlp (dim_hidden*2, dim_hidden*2, 1) --> 52.5 
epoch 50, dropout=0., dim_hidden=400, mlp (dim_hidden*2, dim_hidden*2, 1) --> 52.8 

epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden*2, dim_hidden*2, 1) --> 56.7  
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden*2, dim_hidden*2, 1) --> 54.1 
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden*2, dim_hidden*2, 1) --> 55.1 
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden*2, dim_hidden*2, 1) --> 56.4 
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden*2, dim_hidden*2, 1) --> 55.0 
epoch 60, dropout=0.35, dim_hidden=400, mlp (dim_hidden*2, dim_hidden*2, 1) --> 54.6 

## last to last period 5
epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 53.9
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 56.4
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 56.0
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 57.2
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 55.1
epoch 60, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 53.9

## last to last period 10
epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 50.0
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 57.6
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 58.1
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 54.3
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 52.5
epoch 60, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 49.1

## last to last period 15
epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 50.3, 51.6
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 52.2, 52.8
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 52.8, 54.5
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 52.2, 52.1
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 53.5, 50.7
epoch 60, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 52.3, 52.2
epoch 70, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 51.1, 48.4

## last to last period 20
epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 52.6
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 50.7
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 52.1
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 51.5
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 48.6
epoch 60, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 47.0

## last to pred3 period 5
epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 47.0  
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 56.1  
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 55.0  
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 55.4  
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 56.4  
epoch 60, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 56.3  
epoch 70, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 55.5  
epoch 80, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 56.2  
epoch 90, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 52.6  

## last to pred3 period 10
epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 52.6
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 57.2
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 57.6
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 58.4
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 55.5
epoch 60, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 57.1
epoch 70, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 53.7
epoch 80, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 51.4

## last to pred3 period 15
epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 54.0
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 52.9
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 57.0
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 55.0
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 51.6
epoch 60, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 49.3

## last to pred3 period 20
epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 50.4
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 52.8
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 52.4
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 51.5
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 49.3
epoch 60, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 51.5
epoch 70, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 50.1

## pred3 to last period 5
epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 61.0  
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 60.1  
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 61.9  
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 60.5  
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 61.0  
epoch 60, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 59.5  
epoch 70, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 59.3  
epoch 80, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 54.6  

## pred3 to last period 10
epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 59.0
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 59.1
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 59.8
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 58.6
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 58.7
epoch 60, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 57.9
epoch 70, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 57.5
epoch 80, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 54.1

## pred3 to last period 15
epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 55.9
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 54.9
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 56.6
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 56.0
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 54.9
epoch 60, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 53.3
epoch 70, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 50.5

## pred3 to last    period 20
epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 52.6
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 56.4
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 56.4
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 54.8
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 54.1
epoch 60, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 55.2
epoch 70, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 49.2

In [22]:
rets_2021 = {key: val for key, val in rets.items() if key>'2021-01-01'}
rows = list(rets_2021['2021-01-04'].keys())
cols = list(rets_2021.keys())

In [27]:
rets_2021b = {}
for col in cols:
    for row in rows:
        rets_2021b.setdefault(row, {})
        val = rets_2021[col][row]
        if type(val) is not str:
            rets_2021b[row][col] = rets_2021[col][row].item()        
        else:
            rets_2021b[row][col] = rets_2021[col][row]

In [28]:
pd.DataFrame(rets_2021b).to_csv('./rets.csv', index=True)

In [21]:
rets_2021

{'2021-01-04': {'predicting_date': '2020-12-24',
  'pred': tensor(0.9607),
  'past': tensor(0.9403),
  'tgt': tensor(0.9550)},
 '2021-01-05': {'predicting_date': '2020-12-28',
  'pred': tensor(0.9542),
  'past': tensor(0.9453),
  'tgt': tensor(0.9370)},
 '2021-01-06': {'predicting_date': '2020-12-29',
  'pred': tensor(0.9781),
  'past': tensor(0.9630),
  'tgt': tensor(0.9600)},
 '2021-01-07': {'predicting_date': '2020-12-30',
  'pred': tensor(0.9640),
  'past': tensor(0.9690),
  'tgt': tensor(0.9700)},
 '2021-01-08': {'predicting_date': '2020-12-31',
  'pred': tensor(0.9614),
  'past': tensor(0.9733),
  'tgt': tensor(0.9770)},
 '2021-01-11': {'predicting_date': '2021-01-04',
  'pred': tensor(0.9480),
  'past': tensor(0.9650),
  'tgt': tensor(0.9650)},
 '2021-01-12': {'predicting_date': '2021-01-05',
  'pred': tensor(0.9365),
  'past': tensor(0.9540),
  'tgt': tensor(0.9770)},
 '2021-01-13': {'predicting_date': '2021-01-06',
  'pred': tensor(0.9564),
  'past': tensor(0.9507),
  'tgt': t